# 本单元用于记录LeetCode每日一题

## 问题1：[HTML 实体解析器](https://leetcode.cn/problems/html-entity-parser/description/)
### 题目
字符串替换, 将字符串中的特殊符号, 根据规则转化为指定符号。
### 分析
- 使用正则表达式进行划分, 由于我们发现待替换的字符串中都是“&xxx;”的形式，因此我们使用正则表达式进行分组
- re.split(r"(&[^&;]+;)", text):
    - [^&;]: 表示匹配不包括&和;符号
    - +: 表示前面的匹配至少出现一次
    - [^&;]+: 表示必须匹配一个非&和非;的字符
    - (): 表示将内部匹配到的东西作为一个分组整体返回(保证了分割字符中包含有&;) 
### 优化
- 直接使用replace(目标值, 更换值)进行替换即可


In [ ]:
import re
def process(text):
    word_dict = {"&quot;":"\"","&apos;":"\'","&amp;":"&", "&gt;":">","&lt;":"<","&frasl;":"/"}
    word_list = re.split(r"(&[^&;]+;)", text)
    print(word_list)
    for index, word in enumerate(word_list):
        if word in word_dict:
            word_list[index] = word_dict[word]
    return "".join(word_list)

text = "and I quote: &quot;...&quot;"
process(text)


## 问题2：[确定两个字符串是否接近](https://leetcode.cn/problems/determine-if-two-strings-are-close/description/)
### 题目
如果可以使用以下操作从一个字符串得到另一个字符串，则认为两个字符串接近:
- 交换任意两个现有字符的位置.
- 将某种字符与另一种字符全部互换。
若通过有限步的上述两种操作，可以从原字符串得到另一个字符串则认为两个字符串相近

### 分析
- 由于可以交换任意两个字符位置，因此只要保证原字符串与目标字符串的每种字符个数是一样的即可。
- 又由于可以交换任意两种字符(即把所有的a变成b,同时把所有的b变成a), 因此只用保证原字符串的中字符个数列表与目标字符串字符个数列表具有相同元素即可.(例如[1,2,1] 与 [2,1,1])

### 注意
- 由于两个步骤必须与现有的字符交换，因此如果目标字符串中有字母a，但原字符串中没有字母a那将永远无法转换。

In [ ]:
from collections import Counter

def process(word1, word2):
    ## 得到每个字母计数的数值列表
    count1 = Counter(word1)
    count2 = Counter(word2)

    if set(count1.keys()) != set(count2.keys()):
        return False
    
    count1_list = count1.values()
    count2_list = count2.values()

    ### 判断word的元素个数列表是否相同
    count = {}
    for x in count1_list:
        if x in count:
            count[x] += 1
        else:
            count[x] = 1
    for x in count2_list:
        if x not in count or count[x] == 0:
            return False
        count[x] -= 1

    # 以上代码可以通过以下方式替换(时间复杂度会上升)
    # if sorted(count1_list) != sorted(count2_list):
    #     return False

    return True

word1 = "uau"
word2 = "ssx"
process(word1, word2)


## 问题3：[最小体力消耗路径(并查集)](https://leetcode.cn/problems/path-with-minimum-effort)
一开始你在最左上角的格子 (0, 0) ，且你希望去最右下角的格子 (rows-1, columns-1) （注意下标从 0 开始编号）。你每次可以往 上，下，左，右 四个方向之一移动，你想要找到耗费 体力 最小的一条路径。一条路径耗费的 体力值 是路径上相邻格子之间 高度差绝对值 的 最大值 决定的。

请你返回从左上角走到右下角的最小 体力消耗值 。

### 分析
- 如果使动态规划dp[i][j]表示到i,j位置的最小体力, 先从左到右遍历更新上,左位置，再用右到左遍历更新下，右位置。但实际上这种方式是不对的。因为方向的改变会导致对前面的状态改变，因此不可取。
- 如果把每个一个格子看作是一个地点，那这个题目则是让我们找最短路径，只不过这个最短路径的定义不同。我们可以使用并查集方式，每次放入一个点，当我们加入一条权值为 x 的边之后，如果左上角和右下角从非连通状态变为连通状态，那么 x即为答案(我们先根据权重从小到大排列)。

### 注意
- 联通性判断类型的题目都可以使用并查集


In [ ]:
class UnionFind:
    """"标准并查集对象
    """
    def __init__(self, n: int):
        self.pre = list(range(n))  ## 初始化每一个点都是以自己为主元

    ## 查找函数
    def find(self, x):
        if self.pre[x] == x:
            return x
        else:
            self.pre[x] = self.find(self.pre[x])
            return self.pre[x]

    ## 并函数
    def join(self, x, y):
        fx = self.find(x)
        fy = self.find(y)
        if fx != fy:
            self.pre[fx] = fy # 定义fx的上级为fy

    ## 连通性判断x,y是否联通
    def connected(self, x: int, y: int) -> bool:
        x, y = self.find(x), self.find(y)
        return x == y


def process(heights):
    m, n = len(heights), len(heights[0])
    edges = []  # 构造边对象
    for i in range(m):
        for j in range(n):
            iden = i * n + j # 当前的坐标拉平后的位置
            if i > 0:  ## 由于是双向关系，因此只用保证上下，左右关系即可
                edges.append((iden - n, iden, abs(heights[i][j] - heights[i - 1][j])))
            if j > 0:
                edges.append((iden - 1, iden, abs(heights[i][j] - heights[i][j - 1])))
    edges = sorted(edges, key=lambda x: x[2])
    uf = UnionFind(m * n)  # 初始化并查集
    ans = 0

    for x, y, v in edges:
        uf.join(x, y)
        if uf.connected(0, m * n - 1):
            ans = v
            break
    return ans

## 问题4：[猜数字游戏](https://leetcode.cn/problems/bulls-and-cows/)
给你一个秘密数字 secret 和朋友猜测的数字 guess ，请你返回对朋友这次猜测的提示。提示的格式为 "xAyB" ，x 是公牛个数， y 是奶牛个数，A 表示公牛，B 表示奶牛。请注意秘密数字和朋友猜测的数字都可能含有重复数字。
- 猜测数字中有多少位属于数字和确切位置都猜对了（称为 "Bulls"，公牛），
- 有多少位属于数字猜对了但是位置不对（称为 "Cows"，奶牛）。也就是说，这次猜测中有多少位非公牛数字可以通过重新排列转换成公牛数字。

### 分析
- 维护两个数组true_num和guess_num, true_num[i]表示数字i在secret中出现的次数，guess_num[i]表示数字i在guess中出现的次数
- 同时遍历secret 和 guess的i位置, 如果secret[i] == guess[i]则公牛数字加1, 否则
    - 先进行判断, 当前的正确数字是否已经出现在了guess_num中, 如果是则奶牛数字加1, 同时guess_num对应位置数字出现次数减一
    - 当前的正确数字还没出现在guess_num中，则true_num对应位置+1
    - 再判断猜测数字，是否已经出现在了true_num中, 如果是则奶牛数字加1, 同时true_num对应位置数字出现次数减一
    - 当前的猜测数字还没出现在true_num中，则guess_num对应位置+1


In [ ]:
def process(secret, guess):
    true_nums = [0 for _ in range(10)]
    guess_nums = [0 for _ in range(10)]
    bulls = 0   # 公牛结果
    cows = 0    # 奶牛结果
    for true_num, guess_num in  zip(secret, guess):
        # 判断是否为公牛数字
        if true_num == guess_num:
            bulls += 1
            continue
        
        # 判断是否为奶牛数字
        if guess_nums[int(true_num)] != 0:
            cows += 1
            guess_nums[int(true_num)] -= 1
        else:
            true_nums[int(true_num)] += 1
        
        # 继续判断是否为奶牛数字
        if true_nums[int(guess_num)] != 0:
            cows += 1
            true_nums[int(guess_num)] -= 1
        else:
            guess_nums[int(guess_num)] += 1
        
    return f"{bulls}A{cows}B"

secret = "1123"
guess = "0111"
process(secret, guess)

## 问题4：[]()


### 分析
